# Рекомендательные системы

## Введение


* какие проблемы решает рекомендательная система и как она применяется?
* что такое персональные и неперсональные рекомендации?

Наивный ответ: рекомендательные системы помогают пользователям ориентироваться в "море информации". Честный ответ: рекомендательные системы - это "мягкий" способ продать.

Если проводить аналогии с рекламой: неперсональная рекомендация - это реклама по ТВ, а персонализированная - контекстная реклама.

Рекомендация: https://github.com/microsoft/recommenders

## Наивный пример формирования рекомендации 

Идея рекомендательных систем на основе ближайшего соседа – использовать предпочтения некоторой группы людей, похожих на данного пользователя, чтобы рекомендовать ему что-либо.

У такой модели много применений:
    
   * рекомендация товаров интернет-магазинов
   * рекомендация музыки или фильмов
   * рекомендация людей/контента в социальных сетях

![](amazon.jpg)

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1buQlTY5D-uT18rsjmp-hO60sjtN0rpXg" 
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>


**Пример рекомендательной системы Amazon.com**

![](imdb.png)

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1zEIj_g4A-5assEIxY7UCQpLMsaETmyCi" 
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>


**Рекомендация фильмов от IMDb**

Естественный способ получить рекомендацию о чем-либо - спросить мнение об этом у друзей или любых людей, которым нравится то же, что и вам. Эту идею можно использовать и для машины: для каждого человека алгоритм просматривает большую группу людей и ищет в ней подгруппу с похожим на данного человека вкусом. Далее создается список того, что еще нравится этим людям, а затем человеку рекомендуются предложения из этого списка. Такой алгоритм называется алгоритмом **коллаборативной фильтрации**. 

**Отключаем предупреждения**

Библиотека **warnings** отвечает за то, какие предупреждения (warnings) о работе будут выводиться пользователю. 
FutureWarning - предупреждения о том, как изменится работа библиотек в будущих версиях. Такие предупреждения мы будем игнорировать.
Чтобы включить режим игнорирования, мы отбираем все предупреждения из категории FutureWarning и выбираем для них действия 'ignore'.
Это делается вызовом функции simplefilter c задание двух атрибутов: действия action и категории предупреждений category.

In [3]:
import warnings
warnings.filterwarnings("ignore")

Для красивого вывода на экран сложных объектов будем использовать **pprint**

In [4]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

### Подготовка данных

Для разминки используем пример из книги Т.Сегарана "Программируем коллективный разум".

Чтобы хранить сразу много предпочтений для каждого человека, удобнее всего воспользоваться **вложенным словарем**. 

Самый простой способ создать словарь в python - использовать фигурные скобки {}. Данные в словаре хранятся в формате ключ – значение, разделенные двоеточием:

```python
dict = {'key': 'value'}
```

Мы будем работать с *вложенным словарем* кинокритиков и выставленных ими оценок для небольшого набора данных о фильмах. Построим его следующим образом:

1) для каждого кинокритика создаем словарь оценок фильмов в формате
   ```python
    scores_dict = {
        'film_1': 'score_1', 
        'film_2': 'score_2', 
        ...
    }
   ```
   
2) создаем словарь кинокритиков, где в качестве значений будет соответствующий ему словарь оценок
   ```python
     critics = {
         'name_1': 'scores_dict_1', 
         'name_2': 'scores_dict_2',
         ...
     }
   ```
   
**Code 1:**

In [5]:
# Словарь кинокритиков и выставленных ими оценок
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [6]:
pp.pprint(critics)

{   'Claudia Puig': {   'Just My Luck': 3.0,
                        'Snakes on a Plane': 3.5,
                        'Superman Returns': 4.0,
                        'The Night Listener': 4.5,
                        'You, Me and Dupree': 2.5},
    'Gene Seymour': {   'Just My Luck': 1.5,
                        'Lady in the Water': 3.0,
                        'Snakes on a Plane': 3.5,
                        'Superman Returns': 5.0,
                        'The Night Listener': 3.0,
                        'You, Me and Dupree': 3.5},
    'Jack Matthews': {   'Lady in the Water': 3.0,
                         'Snakes on a Plane': 4.0,
                         'Superman Returns': 5.0,
                         'The Night Listener': 3.0,
                         'You, Me and Dupree': 3.5},
    'Lisa Rose': {   'Just My Luck': 3.0,
                     'Lady in the Water': 2.5,
                     'Snakes on a Plane': 3.5,
                     'Superman Returns': 3.5,
                 

In [7]:
for i in critics.keys():
  print(critics[i])

{'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}
{'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}
{'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0}
{'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5}
{'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}
{'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5}
{'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}


**Code 2:**  
Выведем оценки, которые выставил Toby

In [8]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman Returns': 4.0}

**Code 3:**  
Выведем оценку критика Lisa Rose для фильма Lady in the Water

In [9]:
critics['Lisa Rose']['Lady in the Water']

2.5

Какой здесь могла бы быть неперсонализированная рекомендация? Попробуйте написать свою реализацию неперсонализированной рекомендации.

### Критерий похожести

Как мы уже говорили, чтобы выделить подгруппу людей с похожим вкусом, необходимо как-то определить, насколько люди похожи. В нашем случае мы будем сравнивать оценки людей у одинаковых фильмов. Мы рассмотрим два способа сравнения. С помощью 
- расстояния Евклида 
- корреляции Пирсона

**Оценка по евклидову расстоянию**

Евклидово расстояние для точек $x = (x_1, x_2, ... , x_n)$ и $y = (y_1, y_2, ... , y_n)$ определяется следующим образом:

\begin{equation}
d(x, y) = \sqrt{\sum_{i = 1}^n (x_i - y_i)^2}
\end{equation}

Для понимания расстояния Евклида рассмотрим случай, когда *n = 2*. То есть каждый объект выборки (точка) описывается двумя параметрами (координатами). $x = (x_1, x_2)$ и $y = (y_1, y_2)$. Тогда Евклидово расстояние равно расстоянию между точками на плоскости.

\begin{equation}
d(x, y) = \sqrt{\sum_{i = 1}^n (x_i - y_i)^2} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
\end{equation}

Или, если представить задачу геометрически:

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1BXgfArl4T8N2M4eFur3JrsGloGWDDAlb" 
alt="IMAGE ALT TEXT HERE" width="300" border="0" /></a>


Для вычисления квадратного корня можно воспользоваться готовой функцией **sqrt()** из библиотеки для работы с числами **math**:

**Code 4:**

**Упражнение 1**  
Реализуем функцию для вычисления расстояния Евклида

In [10]:
from math import sqrt

def compute_euclid(x, y):    
  distance = 0

  if len(x) != len(y):
    return distance

  for i in range(len(x)):
    distance += (y[i] - x[i])**2

  return sqrt(distance)



Для примера вычислим расстояния Евклида для близких точек с координатам (1, 1) и (2, 1):

**Code 5:**

In [11]:
compute_euclid([1, 1], [2, 1])

1.0

Теперь вычислим расстояния Евклида для точек, которые находятся далеко (1, 1) и (8, 10):

**Code 6:**

In [12]:
compute_euclid([1, 1], [8, 10])

11.40175425099138

Расстояние, вычисленное по этой формуле, будет тем меньше, чем больше сходство людей (чем ближе точки). Нам же нужна функция, значение которой будет, наоборот, большое, если люди сильно похожи друг на друга. То есть схожесть больше (от 0 до 1), если точки ближе друг другу


Для этого будем использовать функцию "похожести" в таком виде:

\begin{equation}
\text{similarity }(x, y) = \dfrac{1}{1 + d(x, y)},
\end{equation}

где $d(x, y)$ - расстояние Евклида. 

Пример вычисления сходства по новой функции для точек с координатами (1, 1) и (2, 3):

**Code 7:**

In [13]:
1 / (1 + compute_euclid([1, 1], [2, 3]))

0.3090169943749474

Пример вычисления сходства по новой функции для точек с координатами (1, 1) и (8, 10):

**Code 8:**

In [14]:
1 / (1 + compute_euclid([1, 1], [8, 10]))

0.08063375388365411

Далее реализуем несколько простых функций для коллаборативной фильтрации на основе ближайшего соседа. 

**Code 9:**

**Упражнение 2.** Получить список фильмов, оцененных обоими

In [15]:
def similar_films(critics_dict, person1, person2):
    # создаем пустой список фильмов с названием 
    sim_film = []
    # для всех фильмов у person1, если такой фильм оценивался person 2, добавляем этот фильм в список sim_film

    for film in critics_dict[person1]:
        if film in critics_dict[person2]:
            sim_film.append(film)

    return sim_film

Сначала реализуем функцию для подсчета сходства двух критиков на основе расстояния Евклида. Назовём функцию  **sim_distance()**. Функция будет использовать функцию **similar_films()** и принимать на вход три аргумента:
* словарь критиков
* имя первого критика для сравнения
* имя второго критика для сравнения

**Упражнение 3**. Подсчет сходства двух критиков на основе расстояния Евклида

In [16]:

def sim_distance(critics_dict, person1, person2):
    # используем написанную функцию для получения фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    # если нет ни одной общей оценки, то есть длина sim_film равна 0 - вернуть 0
    if len(sim_films) == 0:
        return 0
    
    # для каждого фильма из sim_film посчитаем евклидово расстояние и просуммируем все полученные значения sum_of_euclead_dist
    sum_of_euclead_dist = 0
    for film in sim_films:
        sum_of_euclead_dist = sum_of_euclead_dist + ((critics_dict[person1][film] - critics_dict[person2][film])**2)
    
    # считаем похожесть 
    score = 1 / (1 + sum_of_euclead_dist)

    return score

Посмотрим, как работает эта функция для двух критиков Lisa Rose и Toby:

**Code 10:**

In [17]:
sim_distance(critics, 'Lisa Rose', 'Toby')

0.2222222222222222

Эта функция работает также как:

\begin{equation}
\text{similarity }(x, y) = \dfrac{1}{1 + d(x, y)},
\end{equation}

где $d(x, y)$ - расстояние Евклида. 

Однако, по осям будет фильмы и оценки от критиков. Если у критиков нет одинаковых фильмов, то мы этих критиков не рассматриваем

**Коэффициент корреляции Пирсона** 

Другой способ определять степень схожести - использовать коэффициент корреляции Пирсона. Этот коэффициент показывает, насколько два набора данных хорошо ложатся на прямую. 

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1qXZxG2sE3Y0oAcgayaEdN6nCYVunj_83" 
alt="IMAGE ALT TEXT HERE" width="800" border="0" /></a>

Два кинокритика со средним значением корреляции 0.41 (слева) и два кинокритика с высоким значением корреляции 0.75 (справа).  

Корреляция Пирсона считается по формуле: 
    
\begin{equation}
    \text{corr}(x, y) = \dfrac{\sum_{i = 1}^n (x_i - \frac{\sum_{j = 1}^n x_j}{n}) (y_i - \frac{\sum_{j = 1}^n y_j}{n})}{\sqrt{\sum_{i = 1}^n (x_i - \frac{\sum_{j = 1}^n x_j}{n})^2 \sum_{i = 1}^n (y_i - \frac{\sum_{j = 1}^n y_j}{n})^2}}
\end{equation}

По аналогии с функцией **sim_distance()** реализуйте функцию для подсчета сходства двух критиков на основе корреляции Пирсона. Назовём её  **sim_pearson()**. Функция принимает на вход те же самые аргументы, что и **sim_distance()**:
* словарь критиков
* имя первого критика для сравнения
* имя второго критика для сравнения

Посмотрим, как работает эта функция для двух критиков Lisa Rose и Gene Seymour:

**Code 11:**  
**Упражнение 4.** Допишите функцию для вычисления корреляции Пирсона

In [18]:
def pearson_corr(x, y):
    numerator = 0
    denomirator_x = 0
    denomirator_y = 0

    x_mean = sum(x)/len(x)
    y_mean = sum(y)/len(x)

    for i in range(len(x)):
        numerator = numerator + (x[i] - x_mean)*(y[i] - y_mean)
        denomirator_x = denomirator_x + (x[i] - x_mean)**2
        denomirator_y = denomirator_y + (y[i] - y_mean)**2

    return numerator/ sqrt(denomirator_x * denomirator_y)

Применение без функции **sim_pearson()**:

In [19]:
x = []
y = []
for film in critics['Lisa Rose']:
    x.append(critics['Lisa Rose'][film])
    y.append(critics['Gene Seymour'][film])

print(pearson_corr(x, y))

0.39605901719066977


Это та же функция, но из библиотеки

In [21]:
from scipy.stats import pearsonr
pearsonr(x, y)[0]     # берем [0] от результата, так как функция pearsonr() возвращает еще значение p-value

0.3960590171906697

**Упражнение 5.** Допишите функцию **sim_pearson()**

In [22]:
# Возвращает коэффициент корреляции Пирсона между person1 и person2
def sim_pearson(critics_dict, person1, person2):
    from math import isnan # это нам пригодиться для обработки деления на 0

    # получить список фильмов, оцененных обоими критиками
    sim_films = similar_films(critics_dict, person1, person2)

    # если нет ни одной общей оценки, вернуть 0
    if len(sim_films) < 2:
        return 0
    
    # получим оценки критиков для фильмов из пересечения
    scores1 = []
    scores2 = []
    for film in sim_films:
        scores1.append(critics_dict[person1][film])
        scores2.append(critics_dict[person2][film])

    # посчитаем коэффициент корреляции
    r = pearson_corr(scores1, scores2)

    # если вдруг было деление на ноль и функция вернула nan, то присваиваем ноль
    if isnan(r):
        r = 0
    return r

In [23]:
sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

0.39605901719066977

### Ранжирование критиков

После того как мы выбрали критерий для сравнения двух людей, можно начать искать для каждого человека подгруппу других людей с наиболее похожими вкусами. 

Для этого реализуем функцию **topMatches()**. Функция будет возвращать список n наилучших соответствий для человека из словаря `critics`. 

Соответственно, аргументы у функции: 
* словарь критиков *critics\_dict*
* имя персоны *person*, для которой подбираем соответствия 
* число *n* наилучших соответствий 

Кроме этого, так как мы хотим использовать различные расстояния, то в качестве аргумента мы будем также передавать имя функции-расстояния (*similarity*).  


**Code 12:**

Как работает эта функция: 
*  считаем схожесть интересующего нас критика с каждым другим критиком используя функцию **similarity**
*  сортируем результаты
*  возвращаем топ **n** результатов


**Упражнение 6**. Напишите функцию, которая озвращает топ-n наиболее похожих на person человек из critics_dict

In [24]:

def topMatches(critics_dict, person, n=5, similarity=sim_pearson):
    # инициализируем список оценок похожести
    scores = []

    # для каждого человека в словаре критиков, если этот человек не тот, для которого мы подбираем соответствия
    for user in critics_dict:
        if user != person:
            # добавить в scores похожесть, вычисленную с помощью функции, записанной в similarity, и имя человека
            scores.append((similarity(critics_dict, user, person), user))
            
    # отсортировать список по убыванию оценок
    scores.sort(reverse = True)

    # возвращаем первые n человек    
    return scores[:n]
  

**Code 13:**  
**Упражнение 7.** Вычислите топ-3 критиков похожих на Toby по евклидову расстоянию и по корреляции Пирсона. Они отличаются?

In [25]:
topMatches(critics,'Toby', n=3)

[(0.9912407071619304, 'Lisa Rose'),
 (0.924473451641905, 'Mick LaSalle'),
 (0.8934051474415644, 'Claudia Puig')]

Выбор расстояния для подбора похожести зависит от конкретной задачи. Имеет смысл попробовать разные критерии и посмотреть, какой из них дает наилучший результат. 

## Рекомендация фильмов (User-based подход)

Мы умеем находить людей с похожим мнением как у рассматриваемого человека. Но как получить конкретную рекомендацию с набором фильмов? Можно было бы просто посмотреть на то, какие фильмы понравились похожему человеку, и выбрать из них не просмотренные. Однако, такой способ не стабилен: можем, например, отобрать критика, которому по каким-то причинам понравился фильм, получивший негативные оценки от остальных. 

Чтобы разрешить эту проблему, можно ранжировать сами фильмы, вычисляя взвешенную сумму оценок критиков. Мы будем брать оценку фильма у критика и умножать его на коэффициент похожести с тем, для которого мы строим рекомендацию. Затем полученный результат суммировать и нормировать.  

Идея такого подхода в том, что люди похожие на тебя вносят больший вклад в "рейтинг" фильма. 

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1x0QC1Blvr_ZugJ5BKPJdi2Bp30FvBxvL" 
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>

Давайте реализуем этот подход в функции **getRecommendations()**. Функция принимает на вход следующие аргументы: 
* словарь критиков *critics\_dict*
* имя персоны *person*, для которой подбираем соответствия 
* имя функции-критерия для сравнения


Как работает эта функция:
*  считаем схожесть пользователя с каждым другим пользователем
*  перебираем фильмы, которые смотрели другие пользователи, но не смотрел текущий пользователь
*  нормируем рейтинг фильмов на основе того насколько он понравился другим пользователям, и того насколько каждый из других пользователей похож на текущего пользователя
*  сортируем список

**Code 14:**

In [26]:
# Получить рекомендации для заданного человека, пользуясь взвешенным средним
# оценок, данных всеми остальными пользователями
def getRecommendations(prefs, person, similarity=sim_pearson):

    # инициализируем словари
    totals = {}
    simSums = {}

    # для каждого пользователя
    for other in prefs:
    # сравнивать person с person не нужно
        if other == person:
            continue
        # считаем похожесть
        sim = similarity(prefs, person, other)

        # игнорировать нулевые и отрицательные значения похожести
        if sim <= 0:
            continue

        # для каждого фильма в оцененных other
        for item in prefs[other]:

            # оценивать только фильмы, которые я еще не смотрел
            if item not in prefs[person] or prefs[person][item] == 0:

                # коэффициент подобия * Оценка
                totals.setdefault(item,0)
                totals[item] = totals[item] + prefs[other][item]*sim

                # cумма коэффициентов подобия
                simSums.setdefault(item,0)
                simSums[item] = simSums[item] + sim

    # создать нормированный список
    rankings = [(total/simSums[item], item) for item, total in totals.items( )]

    # вернуть отсортированный список
    rankings.sort(reverse=True)
    return rankings

**Code 15:**

In [27]:
getRecommendations(critics,'Toby', sim_pearson)

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

In [28]:
getRecommendations(critics,'Toby', sim_distance)

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]

Мы получили ранжированный список фильмов, а также прогноз оценки, которую поставит Toby этим фильмам. 

## Метрики качества для рекомендательных систем

Для оценки качества работы рекомендательных систем можно рассмотреть две специальные метрики **Hit Rate** (HR) и **Mean Reciprocal Rank** (MRR).

Определим как **попадание** (**hit**) - ситуацию, когда фильм был рекомендован пользователю в топ-N и пользователь посмотрел его. 

**Hit Rate (HR)**

**Hit Rate (процент попаданий)** определяется как общее число попаданий, нормированное на количество пользователей. 

$$
\text{HR} = \frac{1}{\text{# test users}} \sum_{\text{test users}}{\text{hit}}, \quad
$$

Чем больше процент попаданий, тем лучше будет наша система рекомендаций.

*Замечание*: в некотором смысле, Hit Rate представляет собой производное от классических ML метрик precision и recall. 

**Mean Reciprocal Rank (MRR)**

Отличие метрики **Mean Reciprocal Rank** от предыдущей в том, что мы учитываем не общее количество попаданий, а обратное рангу (месту в списке рекомендаций) **первое попадание**. Например, если фильм, просмотренный пользователем (попадание), стоял на втором месте, обратное рангу будет равно $\frac{1}{2}$, а если бы стоял на третьем - $\frac{1}{3}$ и так далее. 

$$
\text{MRR} = \frac{1}{\text{# test users}} \sum_{\text{test users}}{\frac{1}{\text{hit rank}}}
$$

Рассмотрим на примере. Toby получил следующий список рекомендаций: 

In [29]:
getRecommendations(critics,'Toby', sim_pearson)

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

Допустим, после этого Toby посмотрел 5 фильмов и выставил им следующие оценки:


|Score|Film|
|----|--------------------|
|4.5 | The Departed |
|4   | The Night Listener |
|3.9 | Lady in the Water |
|3.7 | The Firm |
|3   | Just My Luck |

Как мы видим, только два фильма из списка рекомендаций попали в личный топ-3 Toby. Получается, что **попадание (hits)** у него 2.

**HR = $\dfrac{1}{1} * 2 = 2$**, так как выборка тестовых пользователей (test\_users) состоит из одного Toby.  

* **$\frac{1}{hit\_rank}$** для фильма The Night Listener - это $\dfrac{1}{2}$
* **$\frac{1}{hit\_rank}$** для фильма Lady in the Water - это $\dfrac{1}{3}$

Но фильм **The Night Listener** стоит выше в рейтинге Toby, чем **Lady in the Water**, поэтому:

**MRR = $\dfrac{1}{1}*\dfrac{1}{2} = \dfrac{1}{2}$**, так как, опять же, выборка тестовых пользователей (test\_users) состоит из одного Toby.  

## Коллаборативная фильтрация по сходству объектов (Item-based collaborative filtering)

Мы реализовали механизм выдачи рекомендаций таким образом, что для создания набора данных необходимы оценки, выставленные каждым
пользователем. Для нескольких тысяч людей это, возможно, и будет работать, но, когда речь идет о миллионах пользователей и товаров, сравнение каждого пользователя со всеми другими, а затем сравнение товаров, займет недопустимо много времени. 

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1Vd3ofIheUgmxDKLq72cVPTPyl5IyAxo0" 
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>


Техника, которую мы применяли до сих пор, называется **коллаборативной фильтрацией по схожести пользователей**. Альтернатива известна под названием **коллаборативная фильтрация по схожести образцов**. 

Основная идея процедуры фильтрации по схожести образцов заключается в том, чтобы для каждого образца заранее вычислить большинство похожих на него. Тогда для выдачи рекомендаций алгоритму достаточно будет найти те образцы, которым пользователь выставил наивысшие оценки, а затем сформировать список образцов, максимально похожих на эти. 

Отметим одно существенное отличие: хотя на первом шаге необходимо исследовать все данные, результаты сравнения образцов изменяются не так часто, как результаты сравнения пользователей.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1VAoSpaUcbabIFRWlXqrBFDs7LhIISkA5" 
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>

Чтобы сравнивать образцы, нужно первым делом построить полный набор данных о похожих образцах. В первую очередь, нам нужно переделать словарь предпочтений критиков таким образом, чтобы теперь был словарь фильмов и их оценок: 
```python
film_dict = {
    'film_name_1': 
    {'critic_1': 'score_1', 
     'critic_2': 'score_2', 
     ...
    }, 
    ...
}
```
Далее для каждого фильма посчитать наиболее похожие образцы и коэффициенты похожести для них. Для подсчета схожести будем использовать евклидово расстояние. 



In [30]:
# обращаем матрицу предпочтений, чтобы строки соответствовали образцам
def transformPrefs(critics_dict):

    # инициилизируем новый словарь с фильмами и их оценками
    result = {}

    # для каждого критика в словаре
    for person in critics_dict:

        # для каждого оцененного им фильма
        for item in critics_dict[person]:

            # добавляем такой объект в словарь
            result.setdefault(item, {})
            # меняеи местами человека и предмет
            result[item][person] = critics_dict[person][item]
    return result

In [31]:
pp.pprint (transformPrefs (critics))

{   'Just My Luck': {   'Claudia Puig': 3.0,
                        'Gene Seymour': 1.5,
                        'Lisa Rose': 3.0,
                        'Mick LaSalle': 2.0},
    'Lady in the Water': {   'Gene Seymour': 3.0,
                             'Jack Matthews': 3.0,
                             'Lisa Rose': 2.5,
                             'Michael Phillips': 2.5,
                             'Mick LaSalle': 3.0},
    'Snakes on a Plane': {   'Claudia Puig': 3.5,
                             'Gene Seymour': 3.5,
                             'Jack Matthews': 4.0,
                             'Lisa Rose': 3.5,
                             'Michael Phillips': 3.0,
                             'Mick LaSalle': 4.0,
                             'Toby': 4.5},
    'Superman Returns': {   'Claudia Puig': 4.0,
                            'Gene Seymour': 5.0,
                            'Jack Matthews': 5.0,
                            'Lisa Rose': 3.5,
                            'M

**Задание 1.1** Реализуйте функцию **calculateSimilarItems()**, которая делает сразу оба шага. На вход её нужно подать словарь с оценками критиков *critics* и число образцов, которые мы считаем наиболее похожими (остальные не выводим). Например, так `calculateSimilarItems(critics, n=10)`

**Code 16:**

Сначала я написал пример функции calculateSimilarItems используя только расстояние Евклида

In [43]:
# Создать словарь, содержащий для каждого образца те образцы, которые больше всего похожи на него.
def calculateSimilarItems_euclid(prefs, n=10):
    rating = transformPrefs(prefs)

    critics_dict = {}
    i = 0
    for person in prefs:
        critics_dict[person] = i
        i += 1
    
    vectors = {}

    for film in rating:
        vector = [0] * len(critics_dict)

        for critic in rating[film]:
            vector[critics_dict[critic]] = rating[film][critic]

        vectors[film] = vector

    result = {}

    for film in vectors:
        result[film] = []
        similarity_arr = []

        for compare_film in vectors:
            if compare_film != film:
                similarity_arr.append([round(1 / (1 + compute_euclid(vectors[film], vectors[compare_film])), 2), compare_film])

        similarity_arr = sorted(similarity_arr, key = lambda arr: arr[0], reverse = True)

        result[film] = similarity_arr[0 : n]

    return result


In [44]:
itemsim = calculateSimilarItems_euclid(critics, n=3)
pp.pprint(itemsim)

{   'Just My Luck': [   [0.19, 'You, Me and Dupree'],
                        [0.16, 'Lady in the Water'],
                        [0.15, 'The Night Listener']],
    'Lady in the Water': [   [0.21, 'You, Me and Dupree'],
                             [0.17, 'The Night Listener'],
                             [0.16, 'Just My Luck']],
    'Snakes on a Plane': [   [0.31, 'Superman Returns'],
                             [0.17, 'The Night Listener'],
                             [0.16, 'You, Me and Dupree']],
    'Superman Returns': [   [0.31, 'Snakes on a Plane'],
                            [0.17, 'The Night Listener'],
                            [0.15, 'You, Me and Dupree']],
    'The Night Listener': [   [0.17, 'Lady in the Water'],
                              [0.17, 'Snakes on a Plane'],
                              [0.17, 'Superman Returns']],
    'You, Me and Dupree': [   [0.21, 'Lady in the Water'],
                              [0.19, 'Just My Luck'],
                          

Ниже пример с использованием уже написанных выше методов

In [56]:
# Создать словарь, содержащий для каждого образца те образцы, которые больше всего похожи на него.
def calculateSimilarItems(prefs, n=10):
    films_raiting = transformPrefs(prefs)
    similar_films = {}

    for film in films_raiting:
        similar_films[film] = topMatches(films_raiting, film, n, similarity=sim_distance)

    return similar_films 


In [57]:
itemsim = calculateSimilarItems(critics, n=3)
pp.pprint(itemsim)

{   'Just My Luck': [   (0.2222222222222222, 'Lady in the Water'),
                        (0.18181818181818182, 'You, Me and Dupree'),
                        (0.15384615384615385, 'The Night Listener')],
    'Lady in the Water': [   (0.4, 'You, Me and Dupree'),
                             (0.2857142857142857, 'The Night Listener'),
                             (0.2222222222222222, 'Snakes on a Plane')],
    'Snakes on a Plane': [   (0.2222222222222222, 'Lady in the Water'),
                             (0.18181818181818182, 'The Night Listener'),
                             (0.16666666666666666, 'Superman Returns')],
    'Superman Returns': [   (0.16666666666666666, 'Snakes on a Plane'),
                            (0.10256410256410256, 'The Night Listener'),
                            (0.09090909090909091, 'Lady in the Water')],
    'The Night Listener': [   (0.2857142857142857, 'Lady in the Water'),
                              (0.18181818181818182, 'Snakes on a Plane'),
      

Получили вложенный словарь с фильмами и их похожестью на другие фильмы. 

Теперь мы готовы выдавать рекомендации, пользуясь словарем данных о схожести образцов. По аналогии с тем, что мы уже делали, необходимо получить список всех образцов, которым пользователь выставлял оценки, найти похожие и взвесить их с учетом коэффициентов подобия (умножить оценку на коэффициент подобия). 

В таблице ниже приведён приведен рассчёт прогноза оценок для критика Toby. В столбце Фильм и Оценка приведены его оценки за фильмы.  В остальных столбцах приведены коэффициенты похожести для трёх фильмов, которые Toby не оценивал.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1QIJpqC6zQkiQ-1-AzNSmvK4bc8fohsH1" 
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>

Реализуйте этот механихм в функции **getRecommendedItems()**. На вход этой функци подается:
* словарь с оценками пользователя (в нашем случае это словарь критиков, *critics*)
* словарь с данными о схожести образцов, *itemsim*
* имя критика, для которого строим рекомендации

Код, для того, чтобы получить рекомендации для пользователя Toby `getRecommendedItems(critics, itemsim, 'Toby')`

**Code 17:**

In [58]:
def getRecommendedItems(prefs, itemMatch, user):

    # инициализируем словари
    userRatings = prefs[user]
    scores = {}
    totalSim = {}

    # цикл по образцам, оцененным данным пользователем (хранятся в userRatings)
    for (item, rating) in userRatings.items():

        # цикл по образцам, похожим на данный (хранятся в словаре itemMatch)
        for (similarity, item2) in itemMatch[item]:
            # пропускаем объект, если пользователь уже оценивал данный образец
            if item2 in userRatings:
                continue
            # взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item2, 0)
            scores[item2] = scores[item2] + similarity * rating
            # сумма всех коэффициентов подобия
            totalSim.setdefault(item2, 0)
            totalSim[item2] = totalSim[item2] + similarity
            if totalSim[item2] == 0:
                totalSim[item2] = 0.0000001  # чтобы избежать деления на ноль
    # делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить среднее
    rankings = [(score / totalSim[item], item) for item, score in scores.items()]

    # возвращает список rankings, отсортированный по убыванию
    rankings.sort(reverse=True)
    return rankings

In [59]:
getRecommendedItems(critics, itemsim, 'Toby')

[(3.182634730538922, 'The Night Listener'),
 (2.4730878186968837, 'Lady in the Water'),
 (1.0, 'Just My Luck')]

Рекомендация получилась так себе. Ведь это просто ранжированный список фильмов, которые он не смотрел. Давайте в рекомендацию будем вклчать только те фильмы, за которые прогнозируемый средний балл превышает средний балл за те фильмы, которые уже оценил критик. Это второе упражнение на дом. (Первое было про неперсонализированную рекомендацию).

In [60]:
def get_average_score(prefs, user):
    average = 0
    for i in prefs[user]:
        average += prefs[user][i]

    return average/len(prefs[user]) 

def get_recommended_items_avg(prefs, item_match, user):
    recommendations = []
  
    for i in getRecommendedItems(prefs, item_match, user):
        if i[0] > get_average_score(prefs, user):
            recommendations.append(i[1])

    return recommendations


In [61]:
print(get_recommended_items_avg(critics, itemsim, 'Toby'))

['The Night Listener']


## Вопрос на обсуждение

Какие проблемы есть у описанного подхода?